In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "gpuType": "T4"
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "accelerator": "GPU"
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 16,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "mg067nBcg3bV",
        "outputId": "e90fc9e2-110a-40d0-e405-bc10eb261fc4"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Overwriting svg.cu\n"
          ]
        }
      ],
      "source": [
        "%%writefile svg.cu\n",
        "\n",
        "#include <stdio.h>\n",
        "#include <cuda_runtime.h>\n",
        "\n",
        "#define FILTER_WIDTH 3\n",
        "#define TILE_WIDTH 16   // Block size (TILE_WIDTH x TILE_WIDTH)\n",
        "#define IMAGE_SIZE 1024 // Assume a square image (1024x1024)\n",
        "\n",
        "// Example 3x3 edge-detection filter (Sobel)\n",
        "__constant__ float filter[FILTER_WIDTH * FILTER_WIDTH] = {\n",
        "    -1, -1, -1,\n",
        "    -1,  8, -1,\n",
        "    -1, -1, -1\n",
        "};\n",
        "\n",
        "// Global Memory Only Kernel\n",
        "__global__ void convolutionGlobal(float *input, float *output, int width) {\n",
        "    int x = blockIdx.x * blockDim.x + threadIdx.x;\n",
        "    int y = blockIdx.y * blockDim.y + threadIdx.y;\n",
        "\n",
        "    if (x >= width || y >= width) return;\n",
        "\n",
        "    float sum = 0.0;\n",
        "    for (int i = -1; i <= 1; i++) {\n",
        "        for (int j = -1; j <= 1; j++) {\n",
        "            int nx = x + j;\n",
        "            int ny = y + i;\n",
        "            if (nx >= 0 && nx < width && ny >= 0 && ny < width) {\n",
        "                sum += input[ny * width + nx] * filter[(i + 1) * 3 + (j + 1)];\n",
        "            }\n",
        "        }\n",
        "    }\n",
        "    output[y * width + x] = sum;\n",
        "}\n",
        "\n",
        "// Shared Memory Optimized Kernel\n",
        "__global__ void convolutionShared(float *input, float *output, int width) {\n",
        "    __shared__ float tile[TILE_WIDTH + 2][TILE_WIDTH + 2];\n",
        "\n",
        "    int x = blockIdx.x * blockDim.x + threadIdx.x;\n",
        "    int y = blockIdx.y * blockDim.y + threadIdx.y;\n",
        "    int tx = threadIdx.x + 1, ty = threadIdx.y + 1;\n",
        "\n",
        "    // Load shared memory (including halo)\n",
        "    if (x < width && y < width) {\n",
        "        tile[ty][tx] = input[y * width + x];\n",
        "\n",
        "        if (threadIdx.x == 0 && x > 0)\n",
        "            tile[ty][0] = input[y * width + x - 1];\n",
        "        if (threadIdx.x == blockDim.x - 1 && x < width - 1)\n",
        "            tile[ty][tx + 1] = input[y * width + x + 1];\n",
        "        if (threadIdx.y == 0 && y > 0)\n",
        "            tile[0][tx] = input[(y - 1) * width + x];\n",
        "        if (threadIdx.y == blockDim.y - 1 && y < width - 1)\n",
        "            tile[ty + 1][tx] = input[(y + 1) * width + x];\n",
        "    }\n",
        "\n",
        "    __syncthreads();\n",
        "\n",
        "    // Compute convolution\n",
        "    if (x < width && y < width) {\n",
        "        float sum = 0.0;\n",
        "        for (int i = -1; i <= 1; i++)\n",
        "            for (int j = -1; j <= 1; j++)\n",
        "                sum += tile[ty + i][tx + j] * filter[(i + 1) * 3 + (j + 1)];\n",
        "\n",
        "        output[y * width + x] = sum;\n",
        "    }\n",
        "}\n",
        "\n",
        "int main() {\n",
        "    int imageSize = IMAGE_SIZE * IMAGE_SIZE * sizeof(float);\n",
        "    float *h_input, *h_outputGlobal, *h_outputShared;\n",
        "    float *d_input, *d_outputGlobal, *d_outputShared;\n",
        "\n",
        "    // Allocate host memory\n",
        "    h_input = (float*)malloc(imageSize);\n",
        "    h_outputGlobal = (float*)malloc(imageSize);\n",
        "    h_outputShared = (float*)malloc(imageSize);\n",
        "\n",
        "    // Initialize random image\n",
        "    for (int i = 0; i < IMAGE_SIZE * IMAGE_SIZE; i++)\n",
        "        h_input[i] = rand() % 256;\n",
        "\n",
        "    // Allocate device memory\n",
        "    cudaMalloc(&d_input, imageSize);\n",
        "    cudaMalloc(&d_outputGlobal, imageSize);\n",
        "    cudaMalloc(&d_outputShared, imageSize);\n",
        "\n",
        "    cudaMemcpy(d_input, h_input, imageSize, cudaMemcpyHostToDevice);\n",
        "\n",
        "    dim3 blockSize(TILE_WIDTH, TILE_WIDTH);\n",
        "    dim3 gridSize((IMAGE_SIZE + TILE_WIDTH - 1) / TILE_WIDTH, (IMAGE_SIZE + TILE_WIDTH - 1) / TILE_WIDTH);\n",
        "\n",
        "    cudaEvent_t start, stop;\n",
        "    float timeGlobal, timeShared;\n",
        "\n",
        "    // Measure Global Memory Kernel Time\n",
        "    cudaEventCreate(&start);\n",
        "    cudaEventCreate(&stop);\n",
        "    cudaEventRecord(start);\n",
        "    convolutionGlobal<<<gridSize, blockSize>>>(d_input, d_outputGlobal, IMAGE_SIZE);\n",
        "    cudaEventRecord(stop);\n",
        "    cudaEventSynchronize(stop);\n",
        "    cudaEventElapsedTime(&timeGlobal, start, stop);\n",
        "\n",
        "    // Measure Shared Memory Kernel Time\n",
        "    cudaEventRecord(start);\n",
        "    convolutionShared<<<gridSize, blockSize>>>(d_input, d_outputShared, IMAGE_SIZE);\n",
        "    cudaEventRecord(stop);\n",
        "    cudaEventSynchronize(stop);\n",
        "    cudaEventElapsedTime(&timeShared, start, stop);\n",
        "\n",
        "    // Copy results back\n",
        "    cudaMemcpy(h_outputGlobal, d_outputGlobal, imageSize, cudaMemcpyDeviceToHost);\n",
        "    cudaMemcpy(h_outputShared, d_outputShared, imageSize, cudaMemcpyDeviceToHost);\n",
        "\n",
        "    // Print execution times\n",
        "    printf(\"Global Memory Kernel Time: %.3f ms\\n\", timeGlobal);\n",
        "    printf(\"Shared Memory Kernel Time: %.3f ms\\n\", timeShared);\n",
        "    printf(\"Speedup: %.2fx\\n\", timeGlobal / timeShared);\n",
        "\n",
        "    // Cleanup\n",
        "    free(h_input);\n",
        "    free(h_outputGlobal);\n",
        "    free(h_outputShared);\n",
        "    cudaFree(d_input);\n",
        "    cudaFree(d_outputGlobal);\n",
        "    cudaFree(d_outputShared);\n",
        "\n",
        "    return 0;\n",
        "}"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "!nvcc -arch=sm_75 svg.cu -o svg"
      ],
      "metadata": {
        "id": "TLxCjLPXhN6R"
      },
      "execution_count": 17,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "!./svg"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Onk52XwGhh23",
        "outputId": "de992941-f330-43f5-97a9-29f7161ffcc3"
      },
      "execution_count": 22,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Global Memory Kernel Time: 0.340 ms\n",
            "Shared Memory Kernel Time: 0.135 ms\n",
            "Speedup: 2.52x\n"
          ]
        }
      ]
    }
  ]
}